# Assignment 3 – Laplacian Sharpening & Canny Edge Detection
Use this notebook to run all required code.

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread(r"C:\Users\Abdelkader Esmail\Downloads\nature.jpg", cv2.IMREAD_GRAYSCALE)


In [9]:
# Part 1: Laplacian Sharpening
kernel = np.array([[1,1,1],[1,-8,1],[1,1,1]], dtype=np.float32)
laplacian_filtered = cv2.filter2D(img, -1, kernel)
sharpened = cv2.add(img, laplacian_filtered)

plt.figure(figsize=(12,6))
plt.subplot(1,3,1); plt.title("Original"); plt.imshow(img, cmap='gray')
plt.subplot(1,3,2); plt.title("Laplacian Output"); plt.imshow(laplacian_filtered, cmap='gray')
plt.subplot(1,3,3); plt.title("Sharpened"); plt.imshow(sharpened, cmap='gray')
plt.show()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\filter.dispatch.cpp:1527: error: (-215:Assertion failed) !_src.empty() in function 'cv::filter2D'


In [10]:
# OpenCV Laplacian comparison
lap_opencv = cv2.Laplacian(img, cv2.CV_64F, ksize=3)
sharpened_opencv = cv2.convertScaleAbs(img - lap_opencv)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Custom Sharpened"); plt.imshow(sharpened, cmap='gray')
plt.subplot(1,2,2); plt.title("OpenCV Sharpened"); plt.imshow(sharpened_opencv, cmap='gray')
plt.show()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\deriv.cpp:711: error: (-215:Assertion failed) !_src.empty() in function 'cv::Laplacian'


In [11]:
# Part 2: Manual Canny Edge Detection
blur = cv2.GaussianBlur(img, (5,5), 1.4)
Gx = cv2.Sobel(blur, cv2.CV_64F, 1, 0)
Gy = cv2.Sobel(blur, cv2.CV_64F, 0, 1)
magnitude = np.sqrt(Gx**2 + Gy**2)
direction = np.arctan2(Gy, Gx)
magnitude = np.uint8(255 * magnitude / magnitude.max())

def non_max_suppression(mag, dir):
    Z = np.zeros_like(mag)
    angle = dir * 180.0 / np.pi
    angle[angle < 0] += 180
    H, W = mag.shape
    for i in range(1, H-1):
        for j in range(1, W-1):
            q, r = 255, 255
            if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                q = mag[i, j+1]; r = mag[i, j-1]
            elif (22.5 <= angle[i,j] < 67.5):
                q = mag[i+1, j-1]; r = mag[i-1, j+1]
            elif (67.5 <= angle[i,j] < 112.5):
                q = mag[i+1, j]; r = mag[i-1, j]
            else:
                q = mag[i-1, j-1]; r = mag[i+1, j+1]
            Z[i,j] = mag[i,j] if mag[i,j] >= q and mag[i,j] >= r else 0
    return Z

nms = non_max_suppression(magnitude, direction)
low, high = 50, 120
edges = cv2.Canny(img, low, high)

plt.figure(figsize=(14,8))
plt.subplot(2,3,1); plt.title('Original'); plt.imshow(img, cmap='gray')
plt.subplot(2,3,2); plt.title('Blur'); plt.imshow(blur, cmap='gray')
plt.subplot(2,3,3); plt.title('Magnitude'); plt.imshow(magnitude, cmap='gray')
plt.subplot(2,3,4); plt.title('NMS'); plt.imshow(nms, cmap='gray')
plt.subplot(2,3,5); plt.title('OpenCV Canny'); plt.imshow(edges, cmap='gray')
plt.show()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\smooth.dispatch.cpp:618: error: (-215:Assertion failed) !_src.empty() in function 'cv::GaussianBlur'
